In [ ]:
from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

In [ ]:
cluster.dashboard_link

In [ ]:
import dask.dataframe as dd
import dask.bag as db
import json

In [ ]:
b = db.read_text('data/*.json').map(json.loads)
b

In [ ]:
b.take(1)

In [ ]:
ddf = b.to_dataframe()
ddf

In [ ]:
type(ddf)

In [ ]:
df = ddf.compute()
df

In [ ]:
type(df)

In [ ]:
ddf.loc[ddf['occupation'] == "Technician"].compute()

In [ ]:
ddf.drop("name", axis="columns").groupby('occupation').agg({'age': 'mean'}).compute()

## Dataset partitions

In [ ]:
ddf.partitions[0].compute()

In [ ]:
ddf.map_partitions(lambda x: x.size).compute()

# Read CSVs

In [ ]:
from google.cloud import storage

def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client.create_anonymous_client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    # Note: The call returns a response only when the iterator is consumed.
    for blob in blobs:
        print(blob.name)
list_blobs('anaconda-public-data')

In [ ]:
csv_df = dd.read_csv('gcs://anaconda-public-data/nyc-taxi/csv/2015/green_tripdata_2015-03.csv', 
                 storage_options={'token': 'anon'},
                 dtype={'Tolls_amount': 'float64', 'Trip_type ': 'float64'},
                 parse_dates=['lpep_pickup_datetime', 'Lpep_dropoff_datetime']
                )

In [ ]:
csv_df.head()